In [ ]:
from IPython.core.display import display, HTML, Markdown, clear_output, Javascript
from string import Template
import pandas as pd
import json, random
import yaml
import copy
import networkx as nx
import math
import xml.etree.ElementTree as ET
import ipywidgets as widgets
import os
import time
import os.path
from os import path as p
import glob
from datetime import datetime
from IPython.core.display import display, HTML, Markdown, clear_output, Javascript
import ipywidgets as widgets
HTML('''
<script src="../../graph_utils/js/sigma.min.js"></script>
<script src="../../graph_utils/js/sigma.plugins.dragNodes.js"></script>
<script src="../../graph_utils/js/sigma.canvas.edges.labels.curvedArrow.js"></script>
<script src="../../graph_utils/js/sigma.canvas.edges.curvedArrow.js"></script>
<script src="../../graph_utils/js/sigma.canvas.edges.curve.js"></script>
<script src="../../graph_utils/js/sigma.canvas.edges.labels.curve.js"></script>
<script src="../../graph_utils/js/sigma.canvas.edges.labels.def.js"></script>
<script src="../../graph_utils/js/sigma.exporters.svg.js"></script>
<script src="../../graph_utils/js/settings.js"></script>
<script src="../../graph_utils/js/sigma.renders.snapshots.js"></script>
''')

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
import os
from IPython.core.display import display, HTML, Markdown, clear_output, Javascript
import ipywidgets as widgets
from IPython.display import IFrame, display

def on_change(change): # si attiva quando cambio scelta dell'elemento da selezionare
    global attach
    if change['type'] == 'change' and change['name'] == 'value':
        #print ("Stai scegliendo %s" % change['new'])
        attach = change['new']
        return attach

def on_button_update(_):
    clear_output()
    #display(Javascript('''var c = IPython.notebook.get_selected_index();
                       #IPython.notebook.execute_cells([c])'''))
    allegato()

def open_attachment(attach_name):
    path_attachments = "allegati"
    path_file=str(path_attachments + "/" + attach_name)
    ext = os.path.splitext(path_file)[-1].lower()
    #print(path_file)
    #print(ext)
    #print(ext.replace('.', ''))
    if(ext=='.pdf'):
        pdf=IFrame(path_file, width=700, height=400)
        display(pdf)
    else:
        #file = open(path_file, "rb")
        display(Markdown("![alt text](" + path_file + ")"))
        delete = widgets.Button(description='Rimuovi allegato')
        delete.on_click(on_button_delete)
        display(delete)

        #image = file.read()
        #w=widgets.Image(
        #    value=image,
        #    format=ext.replace('.', ''),
        #    width=300,
        #    height=400,
        #)
        #display(w)

def on_button_confirm(_):
    global attach
    clear_output()
    print('Allegato: ' + str(attach))
    open_attachment(attach)

def allegato():
    path_attachments = "allegati"
    global attach # allegato attuale scelto
    update = widgets.Button(description='Aggiorna') #bottone visualizzato in ogni caso
    delete = widgets.Button(description='Rimuovi allegato')
    attach_list = os.listdir(path_attachments)
    if 'ck_points' in attach_list:
        attach_list.remove('ck_points')

    if(attach_list): #si attiva se c'è almeno un elemento in lista quindi nella cartella allegati
        confirm = widgets.Button(description='Conferma')
        chosen_attach_wid = widgets.Dropdown(
            options = attach_list,
            value = attach_list[0],
            description = 'Allegati:',
            disabled = False,
        )
        attach = attach_list[0]
        chosen_attach_wid.observe(on_change)
        update.on_click(on_button_update)
        confirm.on_click(on_button_confirm)
        delete.on_click(on_button_delete)
        all_buttons = [update, chosen_attach_wid, confirm, delete]
        display(widgets.HBox(all_buttons))
    else:
        print(f"ATTENZIONE: non sono presenti file nella cartella allegati.") #Ti ricordiamo che la cartella utile per gli allegati da te prodotti per questo esercizio in modo_libero di svolgimento è la cartella @path_ex_folder@/modo_libero/allegati/")
        update.on_click(on_button_update)
        delete.on_click(on_button_delete)
        all_buttons = [update, delete]
        display(widgets.HBox(all_buttons))

def on_button_delete(_):
    #delete_above_cell()
    delete_this_cell()

def delete_above_cell():
    display(Javascript('''var c = IPython.notebook.get_selected_index();
                                IPython.notebook.get_cell(c-1).metadata.editable = true;
                                IPython.notebook.get_cell(c-1).metadata.deletable = true;
                                IPython.notebook.delete_cell([c-1]);'''))
def delete_this_cell():
    display(Javascript('''var c = IPython.notebook.get_selected_index();
                                IPython.notebook.get_cell(c).metadata.editable = true;
                                IPython.notebook.get_cell(c).metadata.deletable = true;
                                IPython.notebook.delete_cell([c]);'''))

def on_button_raw_attach(_):
    add_attach_cell_din('allegato()')

def on_button_md(_):
    add_md_cell_din()

def on_button_code(_):
    add_code_cell_din()

def on_button_raw(_):
    add_raw_cell_din()

def add_raw_cell_din():
    display_id = int(time.time()*1e9) # Hack
    display(Javascript('''var idx = IPython.notebook.get_selected_index();
                       var c = IPython.notebook.insert_cell_at_index("raw", idx);
    c.set_text('');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_raw(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.get_cell(t_index).metadata.deletable = true;
    IPython.notebook.get_cell(t_index).set_text('');;'''),display_id=display_id) # Hack
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_attach_cell_din(code = ''):
    display_id = int(time.time()*1e9) # Hack
    display(Javascript('''var c = IPython.notebook.insert_cell_above();
    c.set_text(' ''' + code + ''' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_code(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.execute_cells([t_index]);
    IPython.notebook.get_cell(t_index).metadata.editable = false;'''),display_id=display_id) # Hack
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_code_cell_din(code = ''):
    display_id = int(time.time()*1e9)
    display(Javascript('''var c = IPython.notebook.insert_cell_above();
    c.set_text(' ''' + code + ''' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_code(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.execute_cells([t_index]);
    IPython.notebook.get_cell(t_index).metadata.deletable = true;
    IPython.notebook.get_cell(t_index).set_text('');'''),display_id=display_id)
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_md_cell_din():
    display_id = int(time.time()*1e9)
    display(Javascript('''var c = IPython.notebook.insert_cell_above();
    c.set_text(' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_markdown(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.get_cell(t_index).metadata.deletable = true;
    IPython.notebook.get_cell(t_index).set_text('');'''),display_id=display_id) # Hack
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_cell(code='', position='below', celltype='markdown', is_execute = False):
    """Create a cell in the IPython Notebook.
    code: unicode, Code to fill the new cell with.
    celltype: unicode, Type of cells "code" or "markdown".
    position: unicode, Where to put the cell "below" or "at_bottom"
    is_execute: boolean, To decide if the cell is executed after creation
    """

    # Create a unique id based on epoch time
    display_id = int(time.time()*1e9)

    if is_execute:
        display(Javascript("""
        var basis = IPython.notebook.insert_cell_{0}("{1}");
        basis.set_text(atob("{2}"));
        basis.execute();
        """.format(position, celltype, " ")),display_id=display_id)

    else:
        display(Javascript("""
        var basis = IPython.notebook.insert_cell_{0}("{1}");
        basis.set_text(atob("{2}"));
        """.format(position, celltype, " ")),display_id=display_id)


    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)

def code_button_delete():
    button_delete = widgets.Button(description="Rimuovi l'allegato", tooltip="Seleziona la cella e clicca su Elimina")
    button_delete.on_click(on_button_delete)
    #print("Vuoi eliminare la cella selezionata?")
    display(button_delete)

def loader_main():
    button_raw_attach = widgets.Button(description="Rispondi (Allegato)", tooltip="Collega un file dalla cartella attachments")
    button_md = widgets.Button(description="Rispondi (Markdown)", abstooltip="Aggiungi una cella per scrivere del testo in Markdown")
    button_code = widgets.Button(description="Rispondi (Code)", tooltip="Aggiungi una cella per scrivere del codice in Python")
    button_raw = widgets.Button(description="Rispondi (Raw)", tooltip="Aggiungi una cella per scrivere del testo libero (SCONSIGLIATA: righe troppo lunghe potrebbero fuoriuscire nella rendition, usala solo se non riesci a controllare il Markdown)")
    button_raw_attach.on_click(on_button_raw_attach)
    button_md.on_click(on_button_md)
    button_code.on_click(on_button_code)
    button_raw.on_click(on_button_raw)

    all_buttons = [button_code, button_md, button_raw, button_raw_attach]
    display(widgets.HBox(all_buttons))


# ESERCIZIO: Planarità di un grafo #


Si consideri il grafo $G = (V,E)$ riportato di seguito.

__Richiesta 1 \[20 punti\]__: Fornire un planar embedding di $G$

In [ ]:
buttons1 = widgets.Button(description="Avvia task")
buttonl1 = widgets.Button(description="Carica ultima configurazione")
buttonr1 = widgets.Button(description="Reset task")

bllc1 = True
#display(button2, output2)

def run_all1():
    display(Javascript('''
        function sleep2(milliseconds) {
          var start = new Date().getTime();
          for (var i = 0; i < 1e7; i++) {
            if ((new Date().getTime() - start) > milliseconds){
              break;
            }
          }
        }
        var output_area = this;
		// find my cell element
		var cell_element = output_area.element.parents('.cell');
		// which cell is it?
		var cell_idx = Jupyter.notebook.get_cell_elements().index(cell_element);
        try{
            IPython.notebook.execute_cells([cell_idx+1])
			IPython.notebook.execute_cells([cell_idx+2])
			IPython.notebook.execute_cells([cell_idx+3])
        }catch{}
        '''))
    

def on_run_clicked11(b):
    global bllc1
    bllc1=True
    run_all1()
def on_llc1(b):
    global bllc1
    bllc1=False
    run_all1()

def on_reset_clicked1(b):
    global bllc1
    bllc1=True
    run_all1()    
    
buttons1.on_click(on_run_clicked11)
buttonl1.on_click(on_llc1)
buttonr1.on_click(on_reset_clicked1)
widgets.HBox([buttons1, buttonl1, buttonr1])

In [ ]:

    graph_data1 = {'nodes': [{'id': 'S', 'label': 'S', 'x': -146.5, 'y': -135.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'A', 'label': 'A', 'x': -148.5, 'y': 7.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'F', 'label': 'F', 'x': -34.5, 'y': -211.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'E', 'label': 'E', 'x': -104.5, 'y': -213.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'D', 'label': 'D', 'x': -36.5, 'y': -37.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'C', 'label': 'C', 'x': -106.5, 'y': -39.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'G', 'label': 'G', 'x': 1.5, 'y': -137.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'B', 'label': 'B', 'x': -0.5, 'y': 5.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'H', 'label': 'H', 'x': 119.5, 'y': -135.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'P', 'label': 'P', 'x': 117.5, 'y': 7.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'I', 'label': 'I', 'x': 267.5, 'y': -137.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'Q', 'label': 'Q', 'x': 265.5, 'y': 5.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'W', 'label': 'W', 'x': 373.5, 'y': -135.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'X', 'label': 'X', 'x': 371.5, 'y': 7.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'Z', 'label': 'Z', 'x': 485.5, 'y': -211.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'V', 'label': 'V', 'x': 415.5, 'y': -213.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'U', 'label': 'U', 'x': 483.5, 'y': -37.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'R', 'label': 'R', 'x': 413.5, 'y': -39.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'T', 'label': 'T', 'x': 521.5, 'y': -137.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'Y', 'label': 'Y', 'x': 519.5, 'y': 5.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'N', 'label': 'N', 'x': 161.5, 'y': -37.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'O', 'label': 'O', 'x': 231.5, 'y': -39.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'L', 'label': 'L', 'x': 159.5, 'y': -96.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'M', 'label': 'M', 'x': 229.5, 'y': -98.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'hidden0', 'label': '', 'x': 88.5, 'y': -31.0, 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}, {'id': 'hidden1', 'label': '', 'x': '86.5', 'y': -189.5, 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}, {'id': 'hidden2', 'label': '', 'x': 380.0, 'y': '-191.5', 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}, {'id': 'hidden3', 'label': '', 'x': 8.0, 'y': 40.5, 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}, {'id': 'hidden4', 'label': '', 'x': 274.0, 'y': '38.5', 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}, {'id': 'hidden5', 'label': '', 'x': -28.0, 'y': 61.5, 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}, {'id': 'hidden6', 'label': '', 'x': 300.5, 'y': '59.5', 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}, {'id': 'hidden7', 'label': '', 'x': '272', 'y': 92.5, 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}], 'edges': [{'id': 'e0', 'source': 'E', 'target': 'F', 'type': 'line', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e1', 'source': 'F', 'target': 'G', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e2', 'source': 'E', 'target': 'S', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e3', 'source': 'S', 'target': 'G', 'type': 'line', 'label': '11', 'weight': '11', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e4', 'source': 'E', 'target': 'C', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e5', 'source': 'F', 'target': 'D', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e6', 'source': 'C', 'target': 'A', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e7', 'source': 'D', 'target': 'B', 'type': 'line', 'label': '8', 'weight': '8', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e8', 'source': 'G', 'target': 'B', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e9', 'source': 'C', 'target': 'D', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e10', 'source': 'A', 'target': 'B', 'type': 'arrow', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e11', 'source': 'S', 'target': 'A', 'type': 'arrow', 'label': '15', 'weight': '15', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e12', 'source': 'P', 'target': 'B', 'type': 'arrow', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e13', 'source': 'P', 'target': 'H', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e14', 'source': 'H', 'target': 'I', 'type': 'line', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e15', 'source': 'I', 'target': 'Q', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e16', 'source': 'H', 'target': 'L', 'type': 'line', 'label': '2', 'weight': '2', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e17', 'source': 'I', 'target': 'M', 'type': 'line', 'label': '2', 'weight': '2', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e18', 'source': 'Q', 'target': 'O', 'type': 'line', 'label': '2', 'weight': '2', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e19', 'source': 'P', 'target': 'N', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e20', 'source': 'M', 'target': 'L', 'type': 'arrow', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e21', 'source': 'O', 'target': 'N', 'type': 'arrow', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e22', 'source': 'L', 'target': 'N', 'type': 'line', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e23', 'source': 'M', 'target': 'O', 'type': 'line', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e24', 'source': 'Q', 'target': 'P', 'type': 'arrow', 'label': '11', 'weight': '11', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e25', 'source': 'W', 'target': 'X', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e26', 'source': 'X', 'target': 'Y', 'type': 'arrow', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e27', 'source': 'Y', 'target': 'T', 'type': 'arrow', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e28', 'source': 'X', 'target': 'R', 'type': 'arrow', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e29', 'source': 'R', 'target': 'U', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e30', 'source': 'U', 'target': 'Y', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e31', 'source': 'U', 'target': 'Z', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e32', 'source': 'R', 'target': 'V', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e33', 'source': 'V', 'target': 'Z', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e34', 'source': 'Z', 'target': 'T', 'type': 'arrow', 'label': '9', 'weight': '9', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e35', 'source': 'V', 'target': 'W', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e36', 'source': 'W', 'target': 'T', 'type': 'arrow', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e37', 'source': 'O', 'target': 'W', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e38', 'source': 'G', 'target': 'O', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e39', 'source': 'N', 'target': 'W', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'h0', 'source': 'N', 'target': 'hidden0', 'type': 'line', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h1', 'source': 'hidden0', 'target': 'hidden1', 'type': 'line', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h2', 'source': 'hidden1', 'target': 'hidden2', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h3', 'source': 'hidden2', 'target': 'W', 'type': 'line', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h4', 'source': 'B', 'target': 'hidden3', 'type': 'line', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h5', 'source': 'hidden3', 'target': 'hidden4', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h6', 'source': 'hidden4', 'target': 'Q', 'type': 'line', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h7', 'source': 'B', 'target': 'hidden5', 'type': 'line', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h8', 'source': 'hidden5', 'target': 'hidden6', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h9', 'source': 'hidden6', 'target': 'Q', 'type': 'line', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h10', 'source': 'X', 'target': 'hidden7', 'type': 'line', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h11', 'source': 'hidden7', 'target': 'P', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}]}
    friends_e1 = {'h0': ['h0', 'h1', 'h2', 'h3'], 'h1': ['h0', 'h1', 'h2', 'h3'], 'h2': ['h0', 'h1', 'h2', 'h3'], 'h3': ['h0', 'h1', 'h2', 'h3'], 'h4': ['h4', 'h5', 'h6'], 'h5': ['h4', 'h5', 'h6'], 'h6': ['h4', 'h5', 'h6'], 'h7': ['h7', 'h8', 'h9'], 'h8': ['h7', 'h8', 'h9'], 'h9': ['h7', 'h8', 'h9'], 'h10': ['h10', 'h11'], 'h11': ['h10', 'h11']}
    friends_n1 = {'N': ['N', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'hidden2', 'hidden2', 'W'], 'hidden0': ['N', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'hidden2', 'hidden2', 'W', 'N', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'hidden2', 'hidden2', 'W'], 'hidden1': ['N', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'hidden2', 'hidden2', 'W', 'N', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'hidden2', 'hidden2', 'W'], 'hidden2': ['N', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'hidden2', 'hidden2', 'W', 'N', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'hidden2', 'hidden2', 'W'], 'W': ['N', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'hidden2', 'hidden2', 'W'], 'B': ['B', 'hidden3', 'hidden3', 'hidden4', 'hidden4', 'Q', 'B', 'hidden5', 'hidden5', 'hidden6', 'hidden6', 'Q'], 'hidden3': ['B', 'hidden3', 'hidden3', 'hidden4', 'hidden4', 'Q', 'B', 'hidden3', 'hidden3', 'hidden4', 'hidden4', 'Q'], 'hidden4': ['B', 'hidden3', 'hidden3', 'hidden4', 'hidden4', 'Q', 'B', 'hidden3', 'hidden3', 'hidden4', 'hidden4', 'Q'], 'Q': ['B', 'hidden3', 'hidden3', 'hidden4', 'hidden4', 'Q', 'B', 'hidden5', 'hidden5', 'hidden6', 'hidden6', 'Q'], 'hidden5': ['B', 'hidden5', 'hidden5', 'hidden6', 'hidden6', 'Q', 'B', 'hidden5', 'hidden5', 'hidden6', 'hidden6', 'Q'], 'hidden6': ['B', 'hidden5', 'hidden5', 'hidden6', 'hidden6', 'Q', 'B', 'hidden5', 'hidden5', 'hidden6', 'hidden6', 'Q'], 'X': ['X', 'hidden7', 'hidden7', 'P'], 'hidden7': ['X', 'hidden7', 'hidden7', 'P', 'X', 'hidden7', 'hidden7', 'P'], 'P': ['X', 'hidden7', 'hidden7', 'P']}
    center1 = (112.5, -64.0)
    html_text1 = '''<div>
    <button class="button" id="btn_set_one1">Colora</button>
</div>
<div>
    <button class="button" id="btn_set_two1">Colora</button>
</div>
<div>
    <button class="button" id="btn_unselect_set1">Scolora</button>
</div>
<div>
    <button class="button" id="btn_multiple_sel1">Selezione multipla</button>
</div>
<div>
    <button class="button" id="download_png1">Download Png</button>
</div>
<div>
    <button class="button" id="download_svg1">Download Svg</button>
</div>
<div>
    <button class="button" id="download_adj1">Download Matrice di Adiacenza</button>
</div>
<div>
    <button class="button" id="save1">Salva</button>
</div>'''
    

In [ ]:
#parsing del grafo xml o graphml, viene creato graph_data, original_nodes e original_edges
if bllc1:
    pass
else:
    try:
        list_of_files1 = glob.glob('allegati/ck_points/graph_data_'+str(1)+'_*')
        latest_file1 = max(list_of_files1, key=os.path.getctime)
    except:
        pass
    try:
        with open(latest_file1) as json_file:     
            graph_data1 = json.load(json_file)
    except:
        print("Nessun file di configurazione trovato!")

In [ ]:
js_text_template1 = Template(open('../../graph_utils/js/sigma-graph.js','r').read())
#faccio il dump della struttura che ho creato dentro il js
js_text1 = js_text_template1.substitute({'graph_data': json.dumps(graph_data1),
                                       'exercise' : 1,
                                      'container':'sigma-container1',
                                      'data':0,
                                       'x':center1[0]+center1[0]*0.6,
                                       'y':center1[1],
                                      's':'s1',
                                     'n':'n1',
                                      'ef':'`'+str(friends_e1)+'`',
                                      'nf':'`'+str(friends_n1)+'`',
                                       's_cell': 'cell1'})

html_template1 = Template('''
<style>
    .container { width:100% !important; }
</style>
<link rel="stylesheet" href="../../graph_utils/css/style.css">

    <div class="row row-no-gutters" style="border-color: #1e6add; border-width: 2px;">
        <div class="col-sm-1" style="border-color: #1e6add; border-width: 2px;">
            ''' + html_text1 + '''
        </div>
        <div class="col-sm-11" style="position:border-color: #1e6add; border-width: 2px;">
            <div id="sigma-container1" style="height:750px; width:100%"></div>
        </div>
    </div>

<script> $js_text1 </script>
''')
#render HTML con sostituzione variabili
HTML(html_template1.substitute({'js_text1': js_text1}))

In [ ]:
loader_main()


In [ ]:
import os
import time
from urllib.request import urlopen
import nbformat
from traitlets.config import Config
from nbconvert import HTMLExporter
from nbconvert import RSTExporter, NotebookExporter
from IPython.display import Image, HTML, Javascript
from datetime import datetime

import shutil
last ='preview_last/' #path of last submission
old = 'preview_old/' #path of prevous submissions

#hide alert buttons
def hide_w(w):
    for e in w:
        e.layout.visibility = 'hidden'

#show alert buttons
def show_w(w):
    for e in w:
        e.layout.visibility = 'visible'

def are_you_sure():
    show_w(alert_buttons)

label = widgets.Label(value="Un tuo elaborato era già stato precedentemente prodotto. Vuoi rimpiazzarlo con il presente?")
yes = widgets.Button(description="Si", tooltip="Produce il nuovo elaborato. Esso diviene la versione attuale prendendo il posto del precedente")
no = widgets.Button(description="No")

alert_buttons = [label,yes,no]
hide_w(alert_buttons)

#move all files from last to old and save the new preview
def yes_clicked(b):
    files = os.listdir(last)
    for f in files:
        if f == '.ipynb_checkpoints':
            continue
        if not os.path.isdir('./preview_old'):
            os.mkdir(old)
        shutil.move(last+f, old)
    display(Javascript('IPython.notebook.save_checkpoint();'))
    now = datetime.now() # current date and time
    date_time = now.strftime("%Y-%m-%d-%H-%M-%S")
    hname = 'Esercizio_02.ipynb'[:-6] + '_' + date_time
    ! jupyter nbconvert Esercizio_02.ipynb --to html_embed --output=$hname --output-dir=./preview_last/ --TagRemovePreprocessor.remove_cell_tags='["noexport"]'
    display(Javascript('window.open("./preview_last/' + hname + '.html")'))
    hide_w(alert_buttons)
def no_clicked(b):
    hide_w(alert_buttons)


def generate_preview_HTML(_):
    if not os.path.isdir('./preview_last'):
        os.mkdir(last)
    directory= os.listdir(last)
    if len(directory) <1:
        display(Javascript('IPython.notebook.save_checkpoint();'))
        now = datetime.now() # current date and time
        date_time = now.strftime("%Y-%m-%d-%H-%M-%S")
        hname = 'Esercizio_02.ipynb'[:-6] + '_' + date_time
        ! jupyter nbconvert Esercizio_02.ipynb --to html_embed --output=$hname --output-dir=./preview_last/ --TagRemovePreprocessor.remove_cell_tags='["noexport"]'
        display(Javascript('window.open("./preview_last/' + hname + '.html")'))
    else:
        are_you_sure()


button = widgets.Button(description="Salva & Esporta", tooltip="Esporta il foglio Jupyter in HTML nella cartella preview")
output = widgets.Output()
button.on_click(generate_preview_HTML)
yes.on_click(yes_clicked)
no.on_click(no_clicked)
h_box1 = widgets.HBox([button])
h_box2 = widgets.HBox([label,yes,no])
display(widgets.VBox([h_box1,h_box2]))
